In [13]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# For plotting 
import plotly.express as px
import plotly.graph_objects as go # for dropdowns
import geopandas as gpd 
import json # for plotly

In [98]:
# Collect all the unique forest offset projects in Canada 
# TODO - add in the other registries for compliance from other provinces

# Combine the data
meta_projects = pd.read_csv('../data/offset_registries/forestry_projects_meta.csv')
meta_projects['Lat'] = pd.to_numeric(meta_projects['Lat'], errors='coerce')
meta_projects['Lon'] = pd.to_numeric(meta_projects['Lon'], errors='coerce')

## Exploring Offset Credits in BC Carbon Registry
- How many credits have been issued?
- What proportion of projects?
- Where they all available in the same year? (i.e. does fuel switching protocol begin before Forestry - this will skew issuances)

Color points by project status. Make the size of the point based on the number of credits available.
Make line plots for the credits issued by vintage - why does it change? Could this be an issue in the accounting?

Make separate sheet for issuances and another for retirements. 

In [ ]:
# Load Canada’s provinces (GeoJSON format)
url = "https://github.com/nvkelso/natural-earth-vector/raw/master/geojson/ne_50m_admin_1_states_provinces.geojson"
canada_provinces = gpd.read_file(url)
canada_provinces = canada_provinces[canada_provinces["admin"] == "Canada"]

In [ ]:
# Color mapping 
colors_status = {
    "Active": "forestgreen",
    "Under Validation": "darkblue",
    "Withdrawn": "black",
    'Rejected': "red",
    'Under Development': "skyblue",
    'On Hold': "orange"
}

In [114]:
# Plot a map of Canada with points for each forest offset projects using lat lon data
fig = go.Figure()

# Add provinces of Canada to the map
fig.add_trace(go.Choropleth(
    geojson=json.loads(canada_provinces.to_json()),  # Convert GeoJSON
    locations=canada_provinces['name'],  # Province identifiers
    z=[1] * len(canada_provinces),  # Uniform color for all provinces
    colorscale=[[0, 'lightgrey'], [1, 'lightgrey']], # Make all provinces grey
    showscale=False,  # Hide color scale for provinces
    marker_line_width=1,  # Province border width
    marker_line_color='black',  # Province border color
    hoverinfo='location',  # Show province name on hover - from the locations parameter
    featureidkey="properties.name"  # Ensure this matches with GeoJSON key 
))

# Location of forest offset projects ------------------------
# Apply jitter only if there are multiple coordinates at the same location
jitter_strength = 0.6
lat_lon_counts = meta_projects.groupby(['Lat', 'Lon']).size().reset_index(name='counts')
temp_df = meta_projects.merge(lat_lon_counts, on=['Lat', 'Lon'], how='left')
jittered_lat = temp_df['Lat'].copy()
jittered_lon = temp_df['Lon'].copy()
mask = temp_df['counts'] > 1
jittered_lat[mask] += np.random.uniform(-jitter_strength, jitter_strength, size=mask.sum())
jittered_lon[mask] += np.random.uniform(-jitter_strength, jitter_strength, size=mask.sum())
del temp_df

# Prepare hover text
meta_projects['hover_text'] = (
    "<b>Project Name:</b> " + meta_projects['Project Name'] + "<br>" +
    "<b>Project Proponent:</b> " + meta_projects['Project Proponent'] + "<br>" +
    "<b>Project Type:</b> " + meta_projects['Type'] + "<br>" +
    "<b>Registry:</b> " + meta_projects['Registry'] + "<br>" +
    "<b>Status:</b> " + meta_projects['Status'] + "<br>"     
)

fig.add_trace(go.Scattergeo(
    lat=jittered_lat,
    lon=jittered_lon,
    mode='markers',
    marker=dict(
        size=10,  # Marker size
        color=meta_projects["Status"].map(colors_status),  
        symbol=meta_projects["Voluntary/Compliance"].map({
            "Voluntary": "circle",
            "Compliance": "diamond"
        }),
    ),
    text=meta_projects['hover_text'],  # Use the formatted hover text
    hoverinfo="text",  # Hover information
    name=''
))

# Add a legend for the symbols
for compliance, symbol in {"Voluntary": "circle", "Compliance": "diamond"}.items():
    fig.add_trace(go.Scattergeo(
        lat=[None], lon=[None],
        mode='markers',
        marker=dict(size=10, symbol=symbol, color='grey'),
        name=compliance
    ))

# Add a legend for the color of the points based on the colors_status dict
for status, color in colors_status.items():
    fig.add_trace(go.Scattergeo(
        lat=[None], lon=[None],
        mode='markers',
        marker=dict(size=10, color=color),
        name=status
        ))

# Make the plot size bigger
fig.update_layout(
    title='Forest Offset Projects in Canada',
    width=1000,
    height=800,
    geo=dict(
        scope='north america',
        projection_type='equirectangular',
        center={"lat": 56.1304, "lon": -106.3468},  # Center on Canada
        showland=True,  # Show land
        landcolor='rgba(0, 0, 0, 0)'  # Land color (transparent)
    ),
    hoverlabel=dict(
        bgcolor="white",
        font_size=12,
        font_family="Arial"
    ),
    uirevision='constant'  # Keep hover open when you click on it
)

fig.show()

In [ ]:
# Process the data
root_BCregistry = '../data/offset_registries/BCcarbon_registry/'
issuances = pd.read_csv(f'{root_BCregistry}BCcarbon_registry_issuances.csv')

In [ ]:
# Convert units to numeric after removing , separator
issuances['Units'] = issuances['Units'].str.replace(',', '')
issuances['Units'] = pd.to_numeric(issuances['Units'])

In [ ]:
# Plot the total units of issuances by project type
project_types = issuances['Project Type'].unique()
total_units = issuances.groupby('Project Type')['Units'].sum()
plt.bar(project_types, total_units)
plt.xlabel('Project Type')
plt.ylabel('Total Units')

In [ ]:
# Plot the total number of project issuances by project type
issuances['Project Type'].value_counts().plot(kind='bar')
#issuances['Project Type'].value_counts(normalize=True).plot(kind='bar')
# Add labels and title
plt.xlabel('Project Type')
plt.ylabel('Number Projects')
plt.title('Number of Projects by Project Type (that was issued)')